In [ ]:
pip install bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.0 MB/s eta 0:00:00


In [ ]:
pip install scikit-bio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from skbio import Sequence,DNA
from sklearn.metrics import roc_auc_score,roc_curve,precision_recall_curve,average_precision_score,auc

In [ ]:
df = pd.read_csv("Assign3.tsv",delimiter="\t")

In [ ]:
df

In [ ]:
sns.countplot(data=df,x=df.columns[1])

As we can see that it is a skewed distribution with majority being 0

Using 2 mer frequency encoding

In [ ]:
def dnaAdder(dna,dnaclass,k):
  for i in range(len(df)):
    temp=[]
    s=DNA(df.loc[i,df.columns[0]])
    freqs=s.kmer_frequencies(k,relative=True)
    dna.append(freqs)
    dnaclass.append(df.loc[i,df.columns[1]])


In [ ]:
dna=[]
dna_class=[]
dnaAdder(dna,dna_class,2)

In [ ]:
dna

In [ ]:
dict_dna ={"DNA":dna,"Class":dna_class}

In [ ]:
df_encoded=pd.DataFrame(dict_dna)

In [ ]:
df_encoded

In [ ]:
df_test = pd.DataFrame(df_encoded)

In [ ]:
df_test

Using regular kmer_frequency to encode the data

In [ ]:
df_dna_kmer = pd.DataFrame(dna)

In [ ]:
df_dna_kmer=df_dna_kmer.join(df_test)

In [ ]:
df_dna_kmer

In [ ]:
df_dna_kmer=df_dna_kmer.fillna(0)

In [ ]:
df_dna_kmer.isnull().values.sum()

In [ ]:
X_2 = df_dna_kmer.iloc[:,:-2].values
y_2 = df_dna_kmer.iloc[:,-1].values

In [ ]:
print(X_2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forestClass=RandomForestClassifier(class_weight="balanced")

In [ ]:
from sklearn.model_selection import StratifiedKFold

Skf= StratifiedKFold(n_splits=10,shuffle=True)


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def scores(scoresArr,X,y,model):
    for train_index,test_index in Skf.split(X,y):
      X_train,X_test=X[train_index],X[test_index]
      y_train,y_test=y[train_index],y[test_index]
      model.fit(X_train,y_train)
      y_predForest=model.predict(X_test)
      scores_forest_kmer = f1_score(y_pred=y_predForest,y_true=y_test)
      scoresArr.append(scores_forest_kmer)

In [ ]:
scoresarr_forest_kmer=[]
model_forest_2mer = RandomForestClassifier(class_weight="balanced")
scores(scoresarr_forest_kmer,X_2,y_2,model_forest_2mer)

In [ ]:
scoresarr_forest_kmer

In [ ]:
print(sum(scoresarr_forest_kmer)/len(scoresarr_forest_kmer))

Using RC Kmer frequency for encoding

In [ ]:
df_to_add=pd.DataFrame(dict_dna)

In [ ]:
def dnaAdderRc(dna,dnaclass,freq):
  for i in range(len(df)):
    temp=[]
    s=DNA(df.loc[i,df.columns[0]])
    s=s.reverse_complement()
    freqs=s.kmer_frequencies(freq,relative=True)
    dna.append(freqs)
    dnaclass.append(df.loc[i,df.columns[1]])


In [ ]:
dna_rc=[]
dna_rc_class=[]
dnaAdderRc(dna_rc,dna_rc_class,2)

In [ ]:
dict_rc_dna={"DNA":dna_rc,"Class":dna_rc_class}

In [ ]:
df_dna_rc = pd.DataFrame(dna_rc)

In [ ]:
df_dna_rc

In [ ]:
df_dna_rc=df_dna_rc.fillna(0)

In [ ]:
df_dna_rc=df_dna_rc.join(pd.DataFrame(dict_rc_dna))

In [ ]:
df_dna_rc

In [ ]:
X_rc = df_dna_rc.iloc[:,:-2].values
y_rc = df_dna_rc.iloc[:,-1].values

In [ ]:
scores_rc_forest=[]

model_forest_rc = RandomForestClassifier(class_weight='balanced')

scores(scores_rc_forest,X_rc,y_rc,model_forest_rc)

In [ ]:
scores_rc_forest

In [ ]:
sum(scores_rc_forest)/10

Using 3 Mer frequency Encoding

In [ ]:
dna_3=[]
dna_class_3=[]
dnaAdder(dna_3,dna_class_3,3)

In [ ]:
dict_dna_3 = {"DNA":dna_3,"Class":dna_class_3}

In [ ]:
df_dna_3_mer = pd.DataFrame(dict_dna_3)

In [ ]:
df_dna_3_mer

In [ ]:
df_dna_3mer = pd.DataFrame(dna_3)

In [ ]:
df_dna_3mer = df_dna_3mer.join(df_dna_3_mer)

In [ ]:
df_dna_3mer

In [ ]:
df_dna_3mer=df_dna_3mer.fillna(0)

In [ ]:
X_3 = df_dna_3mer.iloc[:,:-2].values
y_3 = df_dna_3mer.iloc[:,-1].values

In [ ]:
scores_3_forest=[]

model_forest_3mer= RandomForestClassifier(class_weight="balanced")

scores(scores_3_forest,X_3,y_3,model_forest_3mer)

In [ ]:
print(scores_3_forest)

In [ ]:
sum(scores_3_forest)/10

In [ ]:
def draw_cv_roc_curve(classifier, cv, X, y, title='ROC Curve'):
    """
    Draw a Cross Validated ROC Curve.
    Args:
        classifier: Classifier Object
        cv: StratifiedKFold Object: (https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation)
        X: Feature Pandas DataFrame
        y: Response Pandas Series
    Example largely taken from http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
    """
    # Creating ROC Curve with Cross Validation
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    i = 0
    for train, test in cv.split(X, y):
        probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        tprs.append(np.interp(mean_fpr, fpr, tpr))

        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

        i += 1
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Luck', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def draw_cv_pr_curve(classifier, cv, X, y, title='PR Curve'):
    """
    Draw a Cross Validated PR Curve.
    Keyword Args:
        classifier: Classifier Object
        cv: StratifiedKFold Object: (https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation)
        X: Feature Pandas DataFrame
        y: Response Pandas Series

    Largely taken from: https://stackoverflow.com/questions/29656550/how-to-plot-pr-curve-over-10-folds-of-cross-validation-in-scikit-learn
    """
    y_real = []
    y_proba = []

    i = 0
    for train, test in cv.split(X, y):
        probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
        # Compute ROC curve and area the curve
        precision, recall, _ = precision_recall_curve(y[test], probas_[:, 1])

        # Plotting each individual PR Curve
        plt.plot(recall, precision, lw=1, alpha=0.3,
                 label='PR fold %d (AUC = %0.2f)' % (i, average_precision_score(y[test], probas_[:, 1])))

        y_real.append(y[test])
        y_proba.append(probas_[:, 1])

        i += 1

    y_real = np.concatenate(y_real)
    y_proba = np.concatenate(y_proba)

    precision, recall, _ = precision_recall_curve(y_real, y_proba)

    plt.plot(recall, precision, color='b',
             label=r'Precision-Recall (AUC = %0.2f)' % (average_precision_score(y_real, y_proba)),
             lw=2, alpha=.8)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

ROC curve for 3 mer frequency

In [ ]:
draw_cv_roc_curve(classifier=model_forest_3mer,X=X_3,y=y_3,cv=Skf)

ROC curve for RC 2 mer frequency

In [ ]:
draw_cv_roc_curve(classifier=model_forest_rc,X=X_rc,y=y_rc,cv=Skf)

ROC curve for 2 mer frequency

In [ ]:
draw_cv_roc_curve(classifier=model_forest_rc,X=X_2,y=y_2,cv=Skf)